## **Expectation-Maximization (EM) 알고리즘 in 음성 딥러닝**

EM 알고리즘은 **숨겨진 변수(hidden variables)**를 포함하는 확률 모델에서  
**최대 우도 추정(Maximum Likelihood Estimation, MLE)**을 수행하는 방법이다.  
음성 딥러닝에서는 **GMM-HMM 모델 학습, 음향 모델링** 등에 사용된다.

---

#### **1. EM 알고리즘의 기본 개념**
EM 알고리즘은 **두 단계**로 반복적으로 수행된다.

1. **Expectation Step (E-step)**: 숨겨진 변수의 기대값을 계산한다.
2. **Maximization Step (M-step)**: 기대값을 기반으로 매개변수를 최적화한다.

---

#### **2. EM 알고리즘의 대수적 표현**

우리는 **관측된 데이터(Observed Data) $X$**와  
**숨겨진 변수(Hidden Variable) $Z$**를 가정한다.  
목표는 **모델의 매개변수(parameter) $\theta$**를 최적화하여  
**우도(likelihood) $p(X | \theta)$**를 최대화하는 것이다.

---

###### **(1) 초기 목표: 최대 우도 추정 (MLE)**
우리가 찾고자 하는 최적의 매개변수 $\theta^*$는 아래와 같이 정의된다.

$$
\theta^* = \arg\max_{\theta} \log p(X | \theta)
$$

여기서,  
- $\theta$: 모델의 매개변수 (예: GMM에서는 가우시안 평균 $\mu$, 분산 $\Sigma$, 혼합 계수 $\pi$)  
- $p(X | \theta)$: $\theta$를 사용하여 X가 생성될 확률 (우도)  
- $\log p(X | \theta)$: 로그 가능도(log-likelihood)  

하지만, **숨겨진 변수 $Z$가 존재하므로 직접 최적화가 어렵다.**  
이 때문에 EM 알고리즘을 사용하여 해결한다.

---

###### **(2) 기대 우도(lower bound) 수식: Q-함수**
EM 알고리즘은 **완전한 로그 가능도(log-likelihood)**를 고려한 **Q-함수**를 최적화한다.

$$
Q(\theta | \theta^{(t)}) = \mathbb{E}_Z [\log p(X, Z | \theta) | X, \theta^{(t)}]
$$

여기서,  
- $Q(\theta | \theta^{(t)})$: 현재 $\theta^{(t)}$에서 $\theta$를 업데이트하기 위한 기대값  
- $\mathbb{E}_Z[\cdot]$: 숨겨진 변수 $Z$에 대한 기댓값 (Expectation)  
- $p(X, Z | \theta)$: **완전한 데이터 로그 가능도(Complete-data log-likelihood)**  

이 과정은 $\theta^{(t)}$에서 **현재 모델이 숨겨진 변수 $Z$를 어떻게 예측하는지**를 계산하는 과정이다.

---

###### **(3) E-step (기댓값 계산)**
현재 $\theta^{(t)}$를 기반으로 **숨겨진 변수를 고려한 우도 분포**를 계산한다.

$$
p(Z | X, \theta^{(t)}) = \frac{p(X, Z | \theta^{(t)})}{p(X | \theta^{(t)})}
$$

여기서,  
- $p(Z | X, \theta^{(t)})$: 현재 모델에서 숨겨진 변수 $Z$의 확률 분포  
- $p(X, Z | \theta^{(t)})$: $\theta^{(t)}$를 사용한 전체 데이터의 확률 분포  
- $p(X | \theta^{(t)})$: $\theta^{(t)}$를 사용한 관측 데이터 $X$의 확률  

이 확률을 기반으로 **Q-함수를 업데이트**한다.

---

###### **(4) M-step (최적화)**
새로운 $\theta$를 최적화한다.

$$
\theta^{(t+1)} = \arg\max_{\theta} Q(\theta | \theta^{(t)})
$$

이 과정이 **수렴할 때까지 반복**한다.

---

#### **3. EM 알고리즘의 예제: GMM을 이용한 음향 모델 학습**

GMM(Gaussian Mixture Model)은 여러 개의 **가우시안 분포**를 사용하여 **음향 데이터를 모델링**한다.  
예를 들어, 음성 프레임의 **MFCC(feature vector) $\mathbf{x}$**를 GMM으로 모델링한다고 가정하자.

###### **(1) GMM의 혼합 가우시안 모델**
$$
p(\mathbf{x} | \theta) = \sum_{k=1}^{K} \pi_k \mathcal{N}(\mathbf{x} | \mu_k, \Sigma_k)
$$

여기서,  
- $K$: 가우시안 개수 (혼합 성분 수)  
- $\pi_k$: $k$번째 가우시안의 혼합 계수 (가우시안이 선택될 확률, $\sum \pi_k = 1$)  
- $\mathcal{N}(\mathbf{x} | \mu_k, \Sigma_k)$: 평균 $\mu_k$, 공분산 $\Sigma_k$를 가지는 가우시안 분포  

---

###### **(2) E-step: 책임도(responsibility) 계산**
각 가우시안 성분 $k$가 주어진 샘플 $\mathbf{x}^{(i)}$를 생성할 확률을 계산한다.

$$
\gamma_{ik} = \frac{\pi_k \mathcal{N}(\mathbf{x}^{(i)} | \mu_k, \Sigma_k)}{\sum_{j=1}^{K} \pi_j \mathcal{N}(\mathbf{x}^{(i)} | \mu_j, \Sigma_j)}
$$

여기서,  
- $\gamma_{ik}$: 샘플 $\mathbf{x}^{(i)}$가 $k$번째 가우시안에 속할 확률 (책임도)  

---

###### **(3) M-step: 파라미터 업데이트**
$$
\pi_k^{(t+1)} = \frac{1}{N} \sum_{i=1}^{N} \gamma_{ik}
$$

$$
\mu_k^{(t+1)} = \frac{\sum_{i=1}^{N} \gamma_{ik} \mathbf{x}^{(i)}}{\sum_{i=1}^{N} \gamma_{ik}}
$$

$$
\Sigma_k^{(t+1)} = \frac{\sum_{i=1}^{N} \gamma_{ik} (\mathbf{x}^{(i)} - \mu_k)(\mathbf{x}^{(i)} - \mu_k)^T}{\sum_{i=1}^{N} \gamma_{ik}}
$$

이 과정을 반복하여 **GMM의 매개변수**를 최적화한다.

---

#### **4. EM 알고리즘의 의미**
- **E-step**: 숨겨진 변수 $Z$의 분포를 업데이트하여 기대값을 계산한다.
- **M-step**: 기대값을 기반으로 매개변수 $\theta$를 최적화한다.
- 음성 딥러닝에서는 **HMM-GMM 모델 학습, 발화 분할, 음성 모델링**에 사용된다.

---